# Data Analytics

In [4]:
# importing libraries
import requests
import pandas as pd
import numpy as np

## 1. Crawl Dataset

Perform web scraping on Yahoo Finance to obtain daily stock data of Nvidia from 1 January 2021 to 31 December 2023.
- What are the variables of interest?
- How was the data scraped/collected?

In [44]:
# # scrape stock data
# url = 'https://finance.yahoo.com/quote/NVDA/history/?period1=1669334400&period2=1732506788'
# r = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})

In [703]:
# scrape stock data
url = 'https://finance.yahoo.com/quote/NVDA/history/?period1=1609459200&period2=1703980800'
r = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})

In [705]:
# covert html table into dataframe
read_html_pandas_data = pd.read_html(r.text)[0]
read_html_pandas_data

,Date,Open,High,Low,Close Close price adjusted for splits.,Adj Close Adjusted close price adjusted for splits and dividend and/or capital gain distributions.,Volume
0,"Dec 29, 2023",49.81,50.00,48.75,49.52,49.51,389293000
1,"Dec 28, 2023",49.64,49.88,49.41,49.52,49.51,246587000
2,"Dec 27, 2023",49.51,49.68,49.08,49.42,49.41,233648000
3,"Dec 26, 2023",48.97,49.60,48.96,49.28,49.27,244200000
4,"Dec 22, 2023",49.19,49.38,48.47,48.83,48.82,252507000
...,...,...,...,...,...,...,...
761,"Jan 8, 2021",13.36,13.42,13.04,13.28,13.24,292528000
762,"Jan 7, 2021",12.97,13.38,12.89,13.34,13.31,461480000
763,"Jan 6, 2021",13.22,13.24,12.59,12.61,12.58,580424000
764,"Jan 5, 2021",13.10,13.44,13.09,13.40,13.37,322760000


In [707]:
# export dataframe to CSV file
read_html_pandas_data.to_csv('NVDA_stock_data.csv', index=False)

## 2a.  Data Preparation & Cleaning

In [710]:
# create a copy
df = read_html_pandas_data.copy()

In [712]:
# renaming the columns
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [714]:
# the number of rows
print(f'Number of rows: {df.shape[0]}')

# the number of columns
print(f'Number of columns: {df.shape[1]}')

Number of rows: 766
Number of columns: 7


In [716]:
# Checking for nulls in the table
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

There are no null values in the table.

In [719]:
# Check the data type for each row
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766 entries, 0 to 765
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       766 non-null    object
 1   Open       766 non-null    object
 2   High       766 non-null    object
 3   Low        766 non-null    object
 4   Close      766 non-null    object
 5   Adj Close  766 non-null    object
 6   Volume     766 non-null    object
dtypes: object(7)
memory usage: 42.0+ KB


We observe that all of the columns have the data type `object`, even though we would expect the the `Date` column to have `datetime` data type, `Volume` column to have an `int` or `float` data type, and the rest of the columns to have `float` data type.

### Date column

Let's start with the `Date` column. Firstly, we check each row in the column, ensuring that they are all dates but with object data type.

In [723]:
# count the number of rows that cannot be converted to datetime
pd.to_datetime(df['Date'], errors='coerce').isnull().sum()

0

Based on the result above, we can confirm that the rows in the `Date` column are all dates but with object data type (i.e. all rows can be converted to datetime data type).

So the logical next step is to convert the `Date` column's data type into `datetime`,

In [726]:
# convert data type to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

### The rest of the columns

Now, let's take a closer look at the data in the `Open`, `High`, `Low`, `Close`, `Adj Close`, and `Volume` columns.

We begin by checking for non-numeric entries in these columns.

In [733]:
# create a copy of df to only store numerical columns
df_num = df.copy()

# drop the Date column
df_num.drop('Date', axis=1, inplace=True)

# store numeric columns (to be used later)
num_cols = df_num.columns

In [735]:
# create a copy of df
df_bool = df_num.copy()

# indicate the rows in each column that are non-numeric
for col in df_num.columns:
    df_bool[col] = pd.to_numeric(df_num[col], errors='coerce').isnull()

# locate rows in the dataframe that contain at least 1 non-numeric value
ser_bool = df_bool.any(axis=1)

# print an extract of the dataframe with rows that contain at least 1 non-numeric value
df_num = df_num[ser_bool]
display(df_num)

,Open,High,Low,Close,Adj Close,Volume
17,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
81,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
144,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
209,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
275,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
335,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
398,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
467,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
530,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
595,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend


From the result above, we can see the rows that contain a least 1 numeric values. Notice that in every column, there are entries that are non-numerical, which explains the reason that Pandas assigns the `object` data type to these columns.

Before we drop these rows, we will take a look at any other data entries with the same date as these dividend payment / stock split events.

In [737]:
# the list of dates when non-numeric entries are provided
date_non_numeric = list(df.loc[df_num.index, 'Date'])

# display all data in those dates
filetered_df = df.where(df['Date'].isin(date_non_numeric)).dropna()
display(filetered_df)

,Date,Open,High,Low,Close,Adj Close,Volume
17,2023-12-05,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
18,2023-12-05,45.47,46.60,45.27,46.57,46.56,371718000
81,2023-09-06,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
82,2023-09-06,48.44,48.55,46.58,47.06,47.05,468670000
144,2023-06-07,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
145,2023-06-07,38.92,39.50,37.36,37.47,37.46,511998000
209,2023-03-07,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
210,2023-03-07,23.60,24.13,23.24,23.29,23.28,515154000
275,2022-11-30,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend,0.00 Dividend
276,2022-11-30,15.70,16.93,15.60,16.92,16.91,565298000


It seems that, at the dates when dividend payment / stock split event occur, there is also stock price and volume data.

Moreover, I want to take a closer look at the stock split event to check whether the prices before and after the split are consistent.

In [740]:
# extract date of stock split
bool_split = df['Open'].str.contains('Stock Splits')
date_split = df[bool_split]['Date']
date_split

626   2021-07-20
Name: Date, dtype: datetime64[ns]

The split happened on July 20th, 2021. To see whether the prices before and after the split are consistent, we will print a few rows of data, before and after the split date,

In [749]:
# index of the stock split date
idx_split = date_split.index[0]

# print a few rows before and after the split
df.iloc[idx_split-3 : idx_split+4, :]

,Date,Open,High,Low,Close,Adj Close,Volume
623,2021-07-23,19.66,19.70,19.25,19.56,19.52,195672000
624,2021-07-22,19.64,19.89,19.28,19.59,19.56,323826000
625,2021-07-21,18.88,19.53,18.74,19.41,19.37,371017000
626,2021-07-20,4:1 Stock Splits,4:1 Stock Splits,4:1 Stock Splits,4:1 Stock Splits,4:1 Stock Splits,4:1 Stock Splits
627,2021-07-20,18.73,18.84,18.16,18.61,18.58,434687000
628,2021-07-19,17.92,19.04,17.87,18.78,18.74,749060000
629,2021-07-16,19.03,19.16,18.07,18.16,18.13,688224000


The prices before and after the stock split appear to be consistent (i.e. the prices have been adjusted). Therefore, so no further action is required (such as dividing or multipltying the price by 4 for a 4:1 stock split).

Now let's drop these non-numeric rows,

In [753]:
# drop non-numeric rows
df.drop(df_num.index, axis=0, inplace=True)

The next step is to change the data type of these numeric columns to float/integer,

In [756]:
# change data type of numeric columns to 
for col in num_cols:
    df[col] = pd.to_numeric(df[col])

In [758]:
# check data type
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 0 to 765
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       753 non-null    datetime64[ns]
 1   Open       753 non-null    float64       
 2   High       753 non-null    float64       
 3   Low        753 non-null    float64       
 4   Close      753 non-null    float64       
 5   Adj Close  753 non-null    float64       
 6   Volume     753 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 47.1 KB


The data type has been appropriately changed.

For convenience, we will change the order of the dataframe, so that it is ordered based on the `Date` column in ascending order,

In [763]:
df.sort_values('Date', ascending=True, inplace=True)

Use the `Date` column as the index for the dataframe,

In [766]:
# use Date column as the dataframe index
df.set_index('Date', inplace=True)

## 2b. Features Generation
In this step, we are going to generate more features, derived from the Open, High, Low, Close, Adj Close, and Volume features. The features we will create are,
- compute intraday daily return -> (close - open) / open
- compute close-to-close return (also called Daily Return) -> (Close_today - Closer_yesterday)/Close_yesterday
- cumulative returns (for example, assume you purchase the stock on Nov 25, 2022 and you are holding on to it until Nov 22, 2024)
- compute moving averages for the close price
- Compute daily volatility by copmuting the difference between High and Low
- Is there any other way to compute volatility (such as moving window standard deviation)?
- Extract the day from the date

*Note: From this point onward, 'closing price' will be used to refer to adjusted closing price.*

### Intraday Retrurn
It represents the return an investor receives if the stock was purchased at market open and then sold at market close. It is computed using the following formula,

$$Intraday Return = \frac{(Close_t - Open_t)}{Open_t}$$

In [770]:
# compute intraday return
df['Intraday Return'] = (df['Adj Close'] - df['Open'])/df['Open']

### Daily Return
Daily Return or Close-to-Close Return is computed using interday closing prices, here is the formula,

$$Daily Return = \frac{(Close_t - Close_{t-1})}{Close_{t-1}}$$

In [773]:
# compute close-to-close return (i.e. daily return)
df['Daily Return'] = (df['Adj Close'] - df['Adj Close'].shift(1)) / df['Adj Close'].shift(1)

### Cumulative Return
It is the compounded growth of an investment, asusming that all returns are reinvested. It is computed as follows,

$$Cumulative Return_t = \prod_{i=1}^t (1 + r_i) - 1$$

In [776]:
# compute cumulative return
df['Cumulative Return'] = (df['Daily Return'] + 1).cumprod() - 1

### 10-Day Moving Average
The average closing price from the past 10 trading days. The 10-day moving average at time t is,

$$\text{10-Day Moving Average} = \frac{1}{10} \sum_{i=t-9}^{t} Close\_Price_i$$

In [779]:
# compute MA(10)
df['10-Day Moving Average'] = df['Adj Close'].rolling(window=10).mean()

### High-Low Rage
The difference between the day's high and low. It is used to measure intraday volatility.

In [782]:
# compute high-low range
df['High-Low Range'] = df['High'] - df['Low']

### 10-Day Moving Standard Deviation
The standard deviation of the Daily Returns from the past 10 trading days. The 10-day moving standard deviation at time t is,

$$\text{10-Day Moving Std.} = \frac{1}{10} \sum_{i=t-9}^{t} Daily\_Return_i$$

$$\sigma_{t,10} = \sqrt{\frac{1}{10} \sum_{i=0}^{9} (x_{t-i} - \bar{x}_{t,10})^2}$$

In [786]:
# compute 10-day moving std
df['10-day Moving Std'] = df['Daily Return'].rolling(window=10).mean()

### Weekday
Extract the weekday from the date.

In [789]:
# Extract the day from the date
df['Weekday'] = list(pd.Series(df.index).dt.day_name())

## 3. Exploratory Data Analysis (EDA)
In EDA, we try to better understand the dataset

CHANGE DATA SET TO FROM 1 JAN 2021 TO 31 DEC 2023!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

- Compute the mean, sum, range, and other interesting statistics for numeric columns
    - Annualized standard deviation of daily returns
    - Plot cumulative returns
    - The stock price range (highest and lowest adjsuted price)
    - The date of top 10 largest returns
    - The date of top 10 smallest returns
    - The date of top 10 largest magnitude of returns -> see for volatility clustering
        
        Volatility clustering refers to the phenomenon in financial markets where large changes in asset prices (high volatility) tend to be followed by large changes, and small changes (low volatility) tend to be followed by small changes. It can be observed by plotting the squared returned graph.
        
    - Rank the weekdays in terms of average returns -> draw a bar chart to compare
    - Rank the weekdays in terms of total traded volumes -> draw bar chart
    - Rank the weekdays in terms of annualzied standard deviation of daily returns -> draw bar chart
    - Repeat the above, but change the grouping into month rather than weekdays.
    - Potentially do the same comparing the different years
    - Compare the traded volume in non-event day and in dividend payment day
        Create a new field to indicated wheter the day is a divident payment day or not
    - Count the number of upticks (positive returns) and downticks (negative returns) -> to mix things up, you can use percentage rather than count
- Explore distributions of numeric columns using histograms etc.
    - Explore distribution of Returns -> Test for normality, QQ-plot, Histogram
- Explore the relationship between columns using scatter plots, bar charts, etc
    - Relationship between high-low (i.e. intraday valatility) and volume -> scatter plot and compute correlations
    - The larger the negative returns, the larger the volatility (measured in std)? How about when returns are greatly positive?

In [ ]:
# Analyze Daily Return by Weekday to identify day-of-week effects
df.groupby('Weekday')['Daily Return'].mean().plot(kind='bar')

In [ ]:
# plot squared returns to observe volatility clustering
data['Squared_Returns'] = data['Return'] ** 2

# Plot
plt.figure(figsize=(10, 6))
plt.plot(data['Squared_Returns'])
plt.title('Volatility Clustering in Squared Returns')
plt.xlabel('Date')
plt.ylabel('Squared Returns')
plt.show()

In [699]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Intraday Return', 'Daily Return', 'Cumulative Return',
       '10-Day Moving Average', 'High-Low Range', '10-day Moving Std',
       'Weekday'],
      dtype='object')

In [690]:
df

,Open,High,Low,Close,Adj Close,Volume,Intraday Return,Daily Return,Cumulative Return,10-Day Moving Average,High-Low Range,10-day Moving Std,Weekday
Date,,,,,,,,,,,,,
2022-11-25,16.32,16.49,16.17,16.27,16.26,167934000,-0.003676,NaN,NaN,NaN,0.32,NaN,Friday
2022-11-28,16.03,16.36,15.73,15.83,15.81,303741000,-0.013724,-0.027675,-0.027675,NaN,0.63,NaN,Monday
2022-11-29,15.83,15.93,15.52,15.64,15.62,298384000,-0.013266,-0.012018,-0.039360,NaN,0.41,NaN,Tuesday
2022-11-30,15.70,16.93,15.60,16.92,16.91,565298000,0.077070,0.082586,0.039975,NaN,1.33,NaN,Wednesday
2022-12-01,17.00,17.26,16.64,17.14,17.12,470977000,0.007059,0.012419,0.052891,NaN,0.62,NaN,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-18,139.50,141.55,137.15,140.15,140.15,221866000,0.004659,-0.012889,7.619311,145.074,4.40,0.003225,Monday
2024-11-19,141.32,147.13,140.99,147.01,147.01,227834900,0.040263,0.048948,8.041205,145.784,6.14,0.005282,Tuesday
2024-11-20,147.41,147.56,142.73,145.89,145.89,309871700,-0.010311,-0.007619,7.972325,145.812,4.83,0.000446,Wednesday


In [687]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume,Intraday Return,Daily Return,Cumulative Return,10-Day Moving Average,High-Low Range,10-day Moving Std
count,502.000000,502.000000,502.000000,502.000000,502.000000,5.020000e+02,502.000000,501.000000,501.000000,493.000000,502.000000,492.000000
mean,66.308725,67.505976,64.988964,66.317371,66.305339,4.358044e+08,0.001404,0.004847,3.083962,66.047523,2.517012,0.004948
std,40.153013,40.855283,39.222296,40.051879,40.054301,1.616871e+08,0.024736,0.032476,2.461974,39.304750,2.179845,0.009784
min,13.930000,14.260000,13.880000,14.040000,14.030000,1.575936e+08,-0.083704,-0.100047,-0.137146,14.602000,0.320000,-0.024521
25%,38.040000,39.162500,37.397500,37.867500,37.855000,3.341450e+08,-0.013156,-0.015187,1.334563,37.963000,0.980000,-0.001325
50%,48.230000,48.715000,47.555000,48.125000,48.115000,4.131126e+08,0.001556,0.004075,1.960025,47.849000,1.685000,0.004779
75%,102.022500,106.115000,98.397500,102.235000,102.225000,5.038665e+08,0.016472,0.022774,5.323493,102.126000,3.445000,0.011046
max,149.350000,152.890000,146.260000,148.880000,148.880000,1.543911e+09,0.091028,0.243775,8.156212,145.812000,13.350000,0.035427


If need more ideas:
- Add S&P500 index price to compute relative strength or correlation between them.